# Test process_annotations function to check timestamp processing

In [1]:
cd Y:\Inpatient Sensors -Stroke\Data\biostamp_data

Y:\Inpatient Sensors -Stroke\Data\biostamp_data


In [2]:
# Importing the Libraries
import os
import platform
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import pathlib
import pickle #to save files
from itertools import product
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import time
import re
import copy

In [3]:
path = r'Y:\Inpatient Sensors -Stroke\Data\biostamp_data\controls\HC01'

In [ ]:
def process_annotations(path):
#---------------------------------------------------------------------------------------------------------
# Processes raw annotations file to extract start / end timestamps and remove unnecessary data
#
# Inputs:  path - filepath of the subject folder containing annotations.csv
#
# Outputs: df - dataframe containing list of activities and their start / end timestamps
#---------------------------------------------------------------------------------------------------------
    df = pd.read_csv(os.path.join(path, 'annotations.csv'))
    del df['Timestamp (ms)']
    del df['AnnotationId']
    del df['AuthorId']
    
    # subset Activity Recognition data by partially match EventType string
    df = df[df['EventType'].str.match('Activity')]
    del df['EventType']
    df.Value = df.Value.shift(-1)
    df = df.dropna()
    
    # Create Trial column for Value
    sorter = set(df.Value.unique().flatten())
    sorterIndex = dict(zip(sorter, range(len(sorter))))        
    df['Value_Rank'] = df['Value'].map(sorterIndex)
    df['Trial'] = df.groupby('Value')['Start Timestamp (ms)'].rank(ascending=True).astype(int)
    del df['Value_Rank']
    df = df.reset_index(drop=True).set_index('Value')
    
    return df

In [8]:
df = pd.read_csv(os.path.join(path, 'annotations.csv'))
del df['Timestamp (ms)']
del df['AnnotationId']
del df['AuthorId']
df = df[df['EventType'].str.match('Activity')]
df.Value = df.Value.shift(-1)
del df['EventType']
df = df.dropna()
df

,Start Timestamp (ms),Stop Timestamp (ms),Value
148,1510002069639,1510002130051,LYING
150,1510002153976,1510002214893,SITTING
152,1510002250787,1510002311224,STANDING
154,1510002352692,1510002384115,WALKING
156,1510002415166,1510002475751,STANDING
158,1510002535559,1510002547458,STAIRS DOWN
160,1510002568801,1510002581432,STAIRS UP
162,1510002620624,1510002651580,STANDING
164,1510002668365,1510002721676,WALKING
166,1510002852155,1510002863746,WALKING
